In [34]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import random
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from modelling import *
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import EarlyStopping
import pickle
import random
import string
from sklearn.preprocessing import LabelEncoder

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
%load_ext autoreload
%autoreload 2
from modelling import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
data_class  = DataClass(use_prediction= True,use_enhanced="lstm",custom=True)
data_class.create_dataset()
modelling_data = DataModel(data = data_class.data_model,
                           use_enhanced= data_class.use_enhanced,
                           custom_test_index=data_class.enhanced_index,
                           max_len=200,
                           custom=True)
modelling_data.create_padding()
modelling_data.test

modifying features
['link', 'employer', 'occupation', 'name_sex', 'firstname_lower']
['link', 'employer', 'occupation', 'name_sex', 'firstname_lower_lstm']
Shape of training tensor:  (205, 200)
Shape of testing tensor:  (27, 200)


,message,label
18,ch prenom femme angèle,1
24,patron prenom femme bertrande,1
25,ep prenom femme margueritte,0
34,Ep prenom homme maxime,0
65,prenom femme angélique,1
69,metages prenom homme felix,1
82,prenom homme georges,1
104,patron prenom homme giuseppe,1
121,frère Labrye manoeuvre prenom homme francesco,1
126,fils prenom homme magloire,1


In [37]:
freq_name = data_class.freq_name.copy()
freq_name = freq_name[freq_name['total']>1000].sort_values(by = 'total')
data_model_translate = freq_name[["firstname"]]
data_model_translate["Y"] = data_model_translate['firstname']
data_model_translate.columns = ["X","Y"]
data_model_translate

C:\Users\marti\AppData\Local\Temp\ipykernel_2724\415725307.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_model_translate["Y"] = data_model_translate['firstname']


,X,Y
734,stéphane,stéphane
733,raimond,raimond
732,dominica,dominica
731,mathie,mathie
730,sicarie,sicarie
...,...,...
4,françois,françois
3,jeanne,jeanne
2,pierre,pierre
1,jean,jean


In [38]:

def alter_word(word):
    operations = []
    if len(word) < 3:
        possible_ops = ['replace_one', 'delete_one']
    else:
        possible_ops = ['replace_two', 'replace_one', 'delete_one', 'delete_two']

    for op in possible_ops:
        if op == 'replace_one':
            pos = random.randint(0, len(word) - 1)
            new_letter = random.choice(string.ascii_letters)
            new_word = word[:pos] + new_letter + word[pos + 1:]
            new_word = str.lower(new_word)
            operations.append(new_word)

        elif op == 'replace_two':
            pos1, pos2 = random.sample(range(len(word)), 2)
            new_letter1, new_letter2 = random.sample(string.ascii_letters, 2)
            new_word = list(word)
            new_word[pos1], new_word[pos2] = new_letter1, new_letter2
            
            operations.append(str.lower(''.join(new_word)))

        elif op == 'delete_one':
            pos = random.randint(0, len(word) - 1)
            new_word = word[:pos] + word[pos + 1:]
            new_word = str.lower(new_word)
            operations.append(new_word)

        elif op == 'delete_two':
            pos1, pos2 = random.sample(range(len(word)), 2)
            new_word = ''.join(letter for i, letter in enumerate(word) if i not in [pos1, pos2])
            new_word = str.lower(new_word)
            operations.append(new_word)

    return operations

data = []
for ope in tqdm(range(50)):
    for index in range(data_model_translate.shape[0]):
        name,true =data_model_translate.iloc[index]
        altered_name = alter_word(name)   
        out = pd.DataFrame([altered_name,[true]*len(altered_name)]).T
        data.append(out)
    
data_model = pd.concat(data,axis=0)
data_model.columns = ['X','Y']

100%|██████████| 50/50 [00:10<00:00,  4.69it/s]


In [41]:

label_encoder = LabelEncoder()
# Tokenize words
tokenizer = Tokenizer(num_words=500, oov_token="<OOV>",char_level=True)
tokenizer.fit_on_texts(data_model['X'])
X_seq = tokenizer.texts_to_sequences(data_model['X'])
X_padded = pad_sequences(X_seq, padding='post', maxlen=8)

Y_encoded = label_encoder.fit_transform(data_model['Y'])
Y_categorical = to_categorical(Y_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X_padded, Y_categorical, test_size=0.1, random_state=42)
# Assuming your LabelEncoder is named label_encoder and has been fitted

# Saving the LabelEncoder
with open('label_encoder.pickle', 'wb') as handle:
    pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)
model = Sequential([
    Embedding(500, 16, input_length=8),
    Bidirectional(LSTM(100, return_sequences=True)),
    Bidirectional(LSTM(100)),
    Dense(100, activation='relu'),
    Dense(Y_categorical.shape[1], activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

model.summary()



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 8, 16)             8000      
                                                                 
 bidirectional_2 (Bidirecti  (None, 8, 200)            93600     
 onal)                                                           
                                                                 
 bidirectional_3 (Bidirecti  (None, 200)               240800    
 onal)                                                           
                                                                 
 dense_6 (Dense)             (None, 100)               20100     
                                                                 
 dense_7 (Dense)             (None, 735)               74235     
                                                                 
Total params: 436735 (1.67 MB)
Trainable params: 43673

In [42]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(X_train,
                    Y_train,
                    epochs=100,
                    validation_data=(X_test, Y_test),
                    validation_split = 0.25,
                    callbacks = [early_stopping],
                    verbose=2)

Epoch 1/100
4135/4135 - 178s - loss: 3.4588 - accuracy: 0.2612 - val_loss: 1.6727 - val_accuracy: 0.5493 - 178s/epoch - 43ms/step
Epoch 2/100
4135/4135 - 157s - loss: 1.1841 - accuracy: 0.6632 - val_loss: 0.9099 - val_accuracy: 0.7286 - 157s/epoch - 38ms/step
Epoch 3/100
4135/4135 - 139s - loss: 0.7428 - accuracy: 0.7704 - val_loss: 0.6835 - val_accuracy: 0.7818 - 139s/epoch - 34ms/step
Epoch 4/100
4135/4135 - 151s - loss: 0.5761 - accuracy: 0.8122 - val_loss: 0.6033 - val_accuracy: 0.8058 - 151s/epoch - 37ms/step
Epoch 5/100


KeyboardInterrupt: 

In [ ]:
model.save('my_model.h5')
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# Loading the LabelEncoder
with open('label_encoder.pickle', 'rb') as handle:
    loaded_label_encoder = pickle.load(handle)


In [ ]:
# First, we tokenize and pad the new X_test strings as we did with the training data
X_test_new =data_class.data.loc[data_class.enhanced_index]['firstname'].str.lower().values.tolist()
X_test_new_seq = tokenizer.texts_to_sequences(X_test_new)
X_test_new_padded = pad_sequences(X_test_new_seq, padding='post', maxlen=8)

# Then we predict using the trained model
predictions_new = model.predict(X_test_new_padded)
predicted_classes_new_indices = np.argmax(predictions_new, axis=1)
predicted_class_labels_new = label_encoder.inverse_transform(predicted_classes_new_indices)
prediction_new_df = pd.DataFrame({
    'Original_X_test': X_test_new,
    'Predicted_Category': predicted_class_labels_new
})
data_class  = DataClass(use_prediction= True,use_enhanced=True,custom=True)

prediction_new_df['true'] = data_class.data.loc[data_class.enhanced_index]['groundtruth'].values
prediction_new_df
data_class.data.loc[data_class.enhanced_index,'firstname_lower'] = predicted_class_labels_new

In [ ]:
from fuzzywuzzy import process
choices = data_model['Y'].unique().tolist()
all_pred = []
for x in X_test_new:
    pred = process.extract(x,choices= choices,limit=1)[0][0]
    all_pred.append(pred)

In [10]:
data_class  = DataClass(use_prediction= True,use_enhanced="lstm",custom=True)
data_class.create_dataset()
modelling_data = DataModel(data = data_class.data_model,
                           use_enhanced= data_class.use_enhanced,
                           custom_test_index=data_class.enhanced_index,
                           max_len=200,
                           custom=True)
modelling_data.create_padding()
modelling_data.test
data_class.data_model.loc[data_class.enhanced_index]

modifying features
['link', 'employer', 'occupation', 'name_sex', 'firstname_lower']
['link', 'employer', 'occupation', 'name_sex', 'firstname_lower_lstm']
Shape of training tensor:  (205, 200)
Shape of testing tensor:  (27, 200)


,message,label
18,ch prenom femme angèle,1
24,patron prenom femme bertrande,1
25,ep prenom femme margueritte,0
34,Ep prenom homme maxime,0
65,prenom femme angélique,1
69,metages prenom homme felix,1
82,prenom homme georges,1
104,patron prenom homme giuseppe,1
121,frère Labrye manoeuvre prenom homme francesco,1
126,fils prenom homme magloire,1


In [15]:
model_socface = Model(data = modelling_data,model_name= "mlp")
model_socface.fit()
model_socface.predict()
model_socface.compute_results()
model_socface.results

Epoch 1/200
6/6 - 2s - loss: 0.8869 - accuracy: 0.5000 - val_loss: 1.1234 - val_accuracy: 0.3659 - 2s/epoch - 328ms/step
Epoch 2/200
6/6 - 1s - loss: 0.8511 - accuracy: 0.4756 - val_loss: 0.8802 - val_accuracy: 0.3659 - 822ms/epoch - 137ms/step
Epoch 3/200
6/6 - 1s - loss: 0.8272 - accuracy: 0.4817 - val_loss: 0.6632 - val_accuracy: 0.6341 - 1s/epoch - 169ms/step
Epoch 4/200
6/6 - 1s - loss: 0.7706 - accuracy: 0.4817 - val_loss: 0.7910 - val_accuracy: 0.3659 - 1s/epoch - 222ms/step
Epoch 5/200
6/6 - 1s - loss: 0.7610 - accuracy: 0.5122 - val_loss: 0.6670 - val_accuracy: 0.6341 - 1s/epoch - 201ms/step
Epoch 6/200
6/6 - 1s - loss: 0.7046 - accuracy: 0.5061 - val_loss: 0.6381 - val_accuracy: 0.6341 - 1s/epoch - 223ms/step
Epoch 7/200
6/6 - 1s - loss: 0.6581 - accuracy: 0.5915 - val_loss: 0.6178 - val_accuracy: 0.8780 - 1s/epoch - 231ms/step
Epoch 8/200
6/6 - 2s - loss: 0.5149 - accuracy: 0.8841 - val_loss: 0.4111 - val_accuracy: 0.8537 - 2s/epoch - 256ms/step
Epoch 9/200
6/6 - 2s - loss: 

{'accuracy_train': 98.04878234863281,
 'accuracy_test': 74.0740716457367,
 'model': 'mlp',
 'custom': 1,
 'missing_names': 'lstm'}